# VERIFICATION TESTING

# HER2 One Scanner - Aperio NIH

- 5-Fold (80/20) split, No Holdout Set
- Truth = Categorical from Mean of 7 continuous scores 
- Epoch at automatic Stop when loss<.001 change 
- LeNet model, 10 layers, Dropout (0.7)

In [1]:
import numpy as np
import pandas as pd
import random
from keras.callbacks import EarlyStopping
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report
import csv
import cv2
import scipy
import os
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#For single scanner
BASE_PATH = '/home/diam/Desktop/1Scanner_VerificationTest_HER2data/Aperio_NIH/'
#BASE PATH for working from home:
#BASE_PATH = '/home/OSEL/Desktop/HER2_data_categorical/'

batch_size = 32
num_classes = 3


## Get Data - Practice

In [3]:
#This is the version from Ravi's code:

#FDA
#X_FDA = []
#idx_FDA = []
#for index, image_filename in list(enumerate(BASE_PATH)):
#	img_file = cv2.imread(BASE_PATH + '/' + image_filename)
#	if img_file is not None:
		#img_file = smisc.imresize(arr = img_file, size = (600,760,3))
#		img_file = smisc.imresize(arr = img_file, size = (120,160,3))		
#		img_arr = np.asarray(img_file)
#		X_FDA.append(img_arr)
#		idx_FDA.append(index)

#X_FDA = np.asarray(X_FDA)
#idx_FDA = np.asarray(idx_FDA)
#random.seed(rs)
#random_id = random.sample(idx_FDA, len(idx_FDA)/2)
#random_FDA = []
#for i in random_id:
#	random_FDA.append(X_FDA[i])

#random_FDA = np.asarray(random_FDA)

## Get Data - Real

In [4]:
def get_data(folder):
    X = []
    y = []
    filenames = []

    for hclass in os.listdir(folder):
        if not hclass.startswith('.'):
            if hclass in ["1"]:
                label = 1
            else: #label must be 1 or 2
                if hclass in ["2"]:
                    label = 2
                else:
                    label = 3
            for image_filename in os.listdir(folder + hclass):
                filename = folder + hclass + '/' + image_filename
                img_file = cv2.imread(folder + hclass + '/' + image_filename)
                
                if img_file is not None:
                    img_file = scipy.misc.imresize(arr=img_file, size=(120, 160, 3))
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
                    filenames.append(filename)

    X = np.asarray(X)
    y = np.asarray(y)
    z = np.asarray(filenames)
    return X,y,filenames

In [5]:
X, y, z = get_data(BASE_PATH)


#print(X)
#print(y)
#print(z)
print(len(X))
print(len(y))
print(y)
print(len(z))

#INTEGER ENCODE
#https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/
encoder = LabelEncoder()
y_cat = np_utils.to_categorical(encoder.fit_transform(y))
print(y_cat)





/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


241
241
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
241
[[ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 

### Old Code

In [6]:

#encoder = LabelEncoder()
#encoder.fit(y)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

#encoded_y_train = encoder.transform(y_train)
#encoded_y_test = encoder.transform(y_test)

#y_train = np_utils.to_categorical(encoded_y_train)
#y_test = np_utils.to_categorical(encoded_y_test)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Fit Model with K-Fold X-Val

In [8]:
kf = KFold(n_splits = 5, random_state=5, shuffle=True)
print(kf.get_n_splits(y))
print(kf)


#for train_index, test_index in kf.split(y):
#    X_train, X_test = X[train_index], X[test_index]
#    print(train_index, test_index)




5
KFold(n_splits=5, random_state=5, shuffle=True)


In [9]:

oos_y = []
oos_pred = []
fold = 0


for train, test in kf.split(y_cat):
    fold+=1
    print("fold #{}".format(fold))
        
    X_train = X[train]
    y_train = y_cat[train]
    X_test = X[test]
    y_test = y_cat[test]
    
    #encoder = LabelEncoder()
    #encoder.fit(y_test)
    #y_train = np_utils.to_categorical(encoder.transform(y_train))
    #y_test = np_utils.to_categorical(encoder.transform(y_test))
    
    model = Sequential()
    model.add(Lambda(lambda x: x * 1./255., input_shape=(120, 160, 3), output_shape=(120, 160, 3)))
    model.add(Conv2D(32, (3, 3), input_shape=(120, 160, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.7))
    model.add(Dense(3))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=25, verbose=1, mode='auto')
    
    model.fit(
            X,
            y_cat,
            validation_data=(X_test,y_test),
            callbacks=[monitor],
            shuffle=True,
            batch_size=batch_size,
            verbose=1,
            epochs=1000)
        
    pred = model.predict(X_test)
        
    oos_y.append(y_test)
    pred = np.argmax(pred,axis=1)
    oos_pred.append(pred)
    
        #measure the fold's accuracy
    y_compare = np.argmax(y_test,axis=1) #for accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print("Fold Score (accuracy): {}".format(score))
    print(pred)

fold #1
Train on 241 samples, validate on 49 samples
Epoch 1/1000
241/241 [==============================] - 4s 17ms/step - loss: 1.7138 - acc: 0.3859 - val_loss: 0.9687 - val_acc: 0.5714
Epoch 2/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0376 - acc: 0.4896 - val_loss: 0.9750 - val_acc: 0.5714
Epoch 3/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9932 - acc: 0.5187 - val_loss: 0.9224 - val_acc: 0.5714
Epoch 4/1000
241/241 [==============================] - 0s 1ms/step - loss: 1.0867 - acc: 0.5519 - val_loss: 0.9455 - val_acc: 0.5714
Epoch 5/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9695 - acc: 0.5187 - val_loss: 0.8281 - val_acc: 0.7551
Epoch 6/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8645 - acc: 0.6432 - val_loss: 0.9147 - val_acc: 0.5714
Epoch 7/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8241 - acc: 0.6846 - val_loss: 0.9590 - val_acc: 0.5714
Epoch 8/

241/241 [==============================] - 0s 1ms/step - loss: 0.2767 - acc: 0.8838 - val_loss: 0.2585 - val_acc: 0.8571
Epoch 62/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3687 - acc: 0.8216 - val_loss: 0.2287 - val_acc: 0.8571
Epoch 63/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2265 - acc: 0.8963 - val_loss: 0.2092 - val_acc: 0.8776
Epoch 64/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4105 - acc: 0.8423 - val_loss: 0.3383 - val_acc: 0.8367
Epoch 65/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2913 - acc: 0.8506 - val_loss: 0.2061 - val_acc: 0.8571
Epoch 66/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.6867 - acc: 0.8008 - val_loss: 0.3504 - val_acc: 0.8571
Epoch 67/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3072 - acc: 0.8714 - val_loss: 0.2389 - val_acc: 0.8571
Epoch 68/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.2158 - acc: 0.9461 - val_loss: 0.0256 - val_acc: 1.0000
Epoch 122/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0604 - acc: 0.9834 - val_loss: 0.0124 - val_acc: 1.0000
Epoch 123/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0832 - acc: 0.9668 - val_loss: 0.0156 - val_acc: 1.0000
Epoch 124/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0680 - acc: 0.9668 - val_loss: 0.0785 - val_acc: 0.9592
Epoch 125/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9289 - acc: 0.8631 - val_loss: 0.2405 - val_acc: 0.9184
Epoch 126/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0760 - acc: 0.9917 - val_loss: 0.0215 - val_acc: 1.0000
Epoch 127/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0756 - acc: 0.9751 - val_loss: 0.0129 - val_acc: 1.0000
Epoch 128/1000
241/241 [==============================] - 0s 1m

Epoch 8/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9739 - acc: 0.5809 - val_loss: 0.9207 - val_acc: 0.6042
Epoch 9/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8754 - acc: 0.6349 - val_loss: 0.6979 - val_acc: 0.7708
Epoch 10/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9038 - acc: 0.6473 - val_loss: 0.9926 - val_acc: 0.5000
Epoch 11/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.9881 - acc: 0.5436 - val_loss: 0.9661 - val_acc: 0.5208
Epoch 12/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.8125 - acc: 0.6722 - val_loss: 0.8760 - val_acc: 0.6250
Epoch 13/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.9548 - acc: 0.6224 - val_loss: 0.6049 - val_acc: 0.7917
Epoch 14/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.6925 - acc: 0.7552 - val_loss: 0.5517 - val_acc: 0.8333
Epoch 15/1000
241/241 [==============================] - 

Epoch 69/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4605 - acc: 0.8216 - val_loss: 0.2656 - val_acc: 0.8542
Epoch 70/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5209 - acc: 0.7801 - val_loss: 0.3080 - val_acc: 0.8542
Epoch 71/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3837 - acc: 0.8216 - val_loss: 0.2847 - val_acc: 0.8542
Epoch 72/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3089 - acc: 0.8506 - val_loss: 0.2092 - val_acc: 0.8542
Epoch 73/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4787 - acc: 0.7967 - val_loss: 0.2676 - val_acc: 0.8542
Epoch 74/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3416 - acc: 0.8465 - val_loss: 0.2063 - val_acc: 0.8542
Epoch 75/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3169 - acc: 0.8589 - val_loss: 0.2004 - val_acc: 0.8542
Epoch 76/1000
241/241 [==============================] 

...

241/241 [==============================] - 0s 1ms/step - loss: 0.0940 - acc: 0.9710 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 134/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0367 - acc: 0.9917 - val_loss: 0.0030 - val_acc: 1.0000
Epoch 135/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0864 - acc: 0.9710 - val_loss: 0.0096 - val_acc: 1.0000
Epoch 136/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3468 - acc: 0.8963 - val_loss: 0.0100 - val_acc: 1.0000
Epoch 137/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0781 - acc: 0.9751 - val_loss: 0.0049 - val_acc: 1.0000
Epoch 138/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0703 - acc: 0.9710 - val_loss: 0.0037 - val_acc: 1.0000
Epoch 139/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0558 - acc: 0.9793 - val_loss: 0.0044 - val_acc: 1.0000
Epoch 140/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.3568 - acc: 0.9212 - val_loss: 0.0326 - val_acc: 1.0000
Epoch 194/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0745 - acc: 0.9793 - val_loss: 0.0038 - val_acc: 1.0000
Epoch 195/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0552 - acc: 0.9834 - val_loss: 9.7249e-04 - val_acc: 1.0000
Epoch 196/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0355 - acc: 0.9834 - val_loss: 7.1845e-04 - val_acc: 1.0000
Epoch 197/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0555 - acc: 0.9959 - val_loss: 7.7646e-04 - val_acc: 1.0000
Epoch 198/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0589 - acc: 0.9793 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 00198: early stopping
Fold Score (accuracy): 1.0
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 0 0 0 0]
fold #5
Train on 241 samples, val

241/241 [==============================] - 0s 1ms/step - loss: 0.4198 - acc: 0.8299 - val_loss: 0.3051 - val_acc: 0.8542
Epoch 55/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4814 - acc: 0.7925 - val_loss: 0.3249 - val_acc: 0.8542
Epoch 56/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3724 - acc: 0.8340 - val_loss: 0.2753 - val_acc: 0.8542
Epoch 57/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4184 - acc: 0.8216 - val_loss: 0.2955 - val_acc: 0.8542
Epoch 58/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.4029 - acc: 0.8382 - val_loss: 0.6126 - val_acc: 0.7500
Epoch 59/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3947 - acc: 0.8216 - val_loss: 0.2669 - val_acc: 0.8750
Epoch 60/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.3872 - acc: 0.8423 - val_loss: 0.2597 - val_acc: 0.8542
Epoch 61/1000
241/241 [==============================] - 0s 1ms/step 

241/241 [==============================] - 0s 1ms/step - loss: 0.1248 - acc: 0.9419 - val_loss: 0.0430 - val_acc: 1.0000
Epoch 115/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2219 - acc: 0.9336 - val_loss: 2.1060 - val_acc: 0.6875
Epoch 116/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.5233 - acc: 0.8921 - val_loss: 0.0571 - val_acc: 1.0000
Epoch 117/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.1557 - acc: 0.9378 - val_loss: 0.0757 - val_acc: 1.0000
Epoch 118/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.1124 - acc: 0.9585 - val_loss: 0.0326 - val_acc: 1.0000
Epoch 119/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0931 - acc: 0.9793 - val_loss: 0.0376 - val_acc: 1.0000
Epoch 120/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.2120 - acc: 0.9336 - val_loss: 0.1265 - val_acc: 0.9792
Epoch 121/1000
241/241 [==============================] - 0s 1m

241/241 [==============================] - 0s 1ms/step - loss: 0.0697 - acc: 0.9668 - val_loss: 0.0052 - val_acc: 1.0000
Epoch 175/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.3694 - acc: 0.9170 - val_loss: 0.0578 - val_acc: 1.0000
Epoch 176/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0766 - acc: 0.9793 - val_loss: 0.0092 - val_acc: 1.0000
Epoch 177/1000
241/241 [==============================] - 0s 2ms/step - loss: 0.0727 - acc: 0.9668 - val_loss: 0.0073 - val_acc: 1.0000
Epoch 178/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0481 - acc: 0.9834 - val_loss: 0.0030 - val_acc: 1.0000
Epoch 179/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0698 - acc: 0.9668 - val_loss: 0.0065 - val_acc: 1.0000
Epoch 180/1000
241/241 [==============================] - 0s 1ms/step - loss: 0.0541 - acc: 0.9751 - val_loss: 0.0024 - val_acc: 1.0000
Epoch 181/1000
241/241 [==============================] - 0s 2m